In [1]:
import sys
import os
sys.path.append('../../')

%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from skimage.io import imread

from gen.load_data import load_data

In [3]:
from sklearn.utils import shuffle

train_df, valid_df, test_df = load_data('../../data')

train_df = shuffle(train_df)
valid_df = shuffle(valid_df)

print(train_df.head())


                                                  image                   id  \
3870  ../../data/Train/CameraRGB/episode_0004_000299...  episode_0004_000299   
4108                 ../../data/Train/CameraRGB/653.png                  653   
2259  ../../data/Train/CameraRGB/episode_0004_000243...  episode_0004_000243   
1081  ../../data/Train/CameraRGB/episode_0009_000088...  episode_0009_000088   
4676  ../../data/Train/CameraRGB/episode_0013_000188...  episode_0013_000188   

                                                  label  
3870  ../../data/Train/CameraSeg/episode_0004_000299...  
4108                 ../../data/Train/CameraSeg/653.png  
2259  ../../data/Train/CameraSeg/episode_0004_000243...  
1081  ../../data/Train/CameraSeg/episode_0009_000088...  
4676  ../../data/Train/CameraSeg/episode_0013_000188...  


In [4]:
from models.unet import model_unetVGG16

model = model_unetVGG16(3, image_shape=(320, 416, 3), keep_prob=0.5)
model.summary()

/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 416, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 320, 416, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 320, 416, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 160, 208, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [5]:
from gen.datagen import oversample_generator_from_df, balanced_generator_from_df

BATCH_SIZE = 32

model_dir = '../../saved_models/unet/unet_v2/'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

from skimage.io import imread
from gen.datagen import preprocess_label

weights = []
for _, row in train_df.iterrows():
    lbl = preprocess_label(imread(row['label']))
    xd, yd = lbl[:, :, 0].nonzero()
    if xd.shape[0] > 0 and yd.shape[0] > 0:
        weights.append( 1 + (xd.max() - xd.min()) * (yd.max() - yd.min()) )
    else:
        weights.append(1)

weights = np.array(weights)
weights = [float(i)/sum(weights) for i in weights]

train_gen = oversample_generator_from_df(train_df, BATCH_SIZE, (320, 416), weights)
valid_gen = balanced_generator_from_df(valid_df, BATCH_SIZE, (320, 416))


In [8]:
from train import iou_loss
from keras.utils.training_utils import multi_gpu_model
from keras.optimizers import RMSprop, Adam

gpus = 2

model_gpu = multi_gpu_model(model, gpus)
# opt = Adam(1e-6)
opt = RMSprop(0.001)
model_gpu.compile(loss=iou_loss,
              optimizer=opt,
              metrics=['acc'])


In [ ]:
from train import train_nn

m = train_df.shape[0]
history = train_nn(model_gpu, 
                   train_gen, 
                   valid_gen, 
                   training_size=1500, 
                   batch_size=BATCH_SIZE,
                   validation_size=valid_df.shape[0],
                   output_path=model_dir, 
                   epochs=100,
                  gpus = 2)

Epoch 1/100
23/23 [==============================] - 25s 1s/step - loss: 0.2336 - acc: 0.7671 - val_loss: 0.2183 - val_acc: 0.7823

Epoch 00001: val_loss improved from inf to 0.21831, saving model to ../../saved_models/unet/unet_v2//model.hdf5
Epoch 2/100
23/23 [==============================] - 22s 958ms/step - loss: 0.2338 - acc: 0.7669 - val_loss: 0.2165 - val_acc: 0.7841

Epoch 00002: val_loss improved from 0.21831 to 0.21648, saving model to ../../saved_models/unet/unet_v2//model.hdf5
Epoch 3/100
23/23 [==============================] - 22s 957ms/step - loss: 0.2334 - acc: 0.7673 - val_loss: 0.2186 - val_acc: 0.7820

Epoch 00003: val_loss did not improve
Epoch 4/100
23/23 [==============================] - 22s 962ms/step - loss: 0.2331 - acc: 0.7676 - val_loss: 0.2165 - val_acc: 0.7840

Epoch 00004: val_loss did not improve
Epoch 5/100
23/23 [==============================] - 22s 969ms/step - loss: 0.2345 - acc: 0.7662 - val_loss: 0.2181 - val_acc: 0.7825

Epoch 00005: val_loss di

In [ ]:
# # summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# model_gpu.load_weights('../../saved_models/unet/unet_v1/model.hdf5')
model.save('../../saved_models/unet/unet_v2/model_saved.h5')